In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -m -p scipy,numpy,sklearn,tensorflow

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.2
IPython 5.1.0

scipy 0.18.1
numpy 1.11.2
sklearn 0.18
tensorflow 0.10.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)
system     : Darwin
release    : 16.1.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [23]:
from sklearn.datasets import load_digits
digits = load_digits()
from sklearn.preprocessing import normalize
digits_data = digits.data / 16

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits_data, digits.target, test_size=0.1)

In [25]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(n_values=10)
y_train_enc = ohe.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_enc = ohe.fit_transform(y_test.reshape(-1, 1)).toarray()
y_train_enc.shape, y_test_enc.shape

((1617, 10), (180, 10))

In [26]:
# from sklearn.preprocessing import scale, normalize
# X_train_scale = normalize(X_train, axis=0)
# X_test_scale = normalize(X_test, axis=0)
# X_train_scale[0], 
X_train[0]

array([ 0.    ,  0.    ,  0.5   ,  0.875 ,  1.    ,  1.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.875 ,  0.8125,  0.5   ,  0.5   ,
        0.    ,  0.    ,  0.    ,  0.125 ,  1.    ,  0.375 ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.375 ,  1.    ,  0.8125,
        1.    ,  0.8125,  0.    ,  0.    ,  0.    ,  0.1875,  1.    ,
        1.    ,  0.75  ,  1.    ,  0.4375,  0.    ,  0.    ,  0.    ,
        0.25  ,  0.0625,  0.125 ,  0.875 ,  0.375 ,  0.    ,  0.    ,
        0.    ,  0.0625,  0.375 ,  1.    ,  0.6875,  0.    ,  0.    ,
        0.    ,  0.    ,  0.6875,  0.9375,  0.5   ,  0.0625,  0.    ,  0.    ])

In [27]:
import tensorflow as tf

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
xb, yb = mnist.train.next_batch(10)
xb[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [9]:
n_dim = 64
n_classes = 10
learning_rate = 0.001
sd = 1 / np.sqrt(n_dim)

In [36]:
X = tf.placeholder(tf.float32, [None, 64])
X_input = tf.reshape(X, [-1, 8, 8, 1])
Y = tf.placeholder(tf.float32, [None, 10])

W_1 = tf.Variable(tf.random_normal([4, 4, 1, 16], mean=0, stddev=1/np.sqrt(64)))
b_1 = tf.Variable(tf.random_normal([16], mean=0, stddev=1/np.sqrt(64)))
z_1 = tf.nn.conv2d(X_input, W_1, strides=[1, 1, 1, 1], padding='SAME') + b_1
h_1 = tf.nn.relu(z_1)

p_1 = tf.nn.max_pool(h_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
p_1_flat = tf.reshape(p_1, [-1, 4*4*16])

W_2 = tf.Variable(tf.random_normal([4*4*16, 100], mean=0, stddev=1/np.sqrt(4*4*16)))
b_2 = tf.Variable(tf.random_normal([100], mean=0, stddev=1/np.sqrt(4*4*16)))
z_2 = tf.matmul(p_1_flat, W_2) + b_2
h_2 = tf.nn.relu(z_2)

W_3 = tf.Variable(tf.random_normal([100, 10], mean=0, stddev=2/np.sqrt(100)))
b_3 = tf.Variable(tf.random_normal([10], mean=0, stddev=2/np.sqrt(100)))
z_3 = tf.matmul(z_2, W_3) + b_3
y_ = tf.nn.softmax(z_3)

init = tf.initialize_all_variables()

cost_function = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [37]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        _, cost = sess.run([optimizer, cost_function], feed_dict={X: X_train, Y: y_train_enc})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X: X_train, Y: y_train_enc})
            print("step %d, training accuracy %g, %g"%(i, cost, train_accuracy))

    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_enc})
    print("test accuracy %g" % (test_accuracy))

step 0, training accuracy 3918.88, 0.150897
step 100, training accuracy 211.805, 0.972171
step 200, training accuracy 108.51, 0.98825
step 300, training accuracy 70.6638, 0.99196
step 400, training accuracy 50.0006, 0.995671
step 500, training accuracy 37.391, 0.997526
step 600, training accuracy 29.0093, 0.998145
step 700, training accuracy 23.1616, 0.998763
step 800, training accuracy 18.9413, 0.999382
step 900, training accuracy 15.8074, 0.999382
test accuracy 0.994444
